In [3]:
import sqlite3
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import xgboost
import random
import pickle

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [4]:
with open('2025-reg-hack-week-airsenal/datasets/training_dictionary.pkl', 'rb') as f:
    train_dict = pickle.load(f)

with open('2025-reg-hack-week-airsenal/datasets/validation_dictionary.pkl', 'rb') as f:
    val_dict = pickle.load(f)

In [12]:
pos = 'DEF'

train_df = train_dict[pos] #full_dict[pos].sample(frac=0.75, random_state=4)
drop_train = np.sort(random.sample([d[0] for d in np.argwhere([True if t_o == 0 else False for t_o in train_df['out']])], int(0.6*len(np.argwhere([True if t_o == 0 else False for t_o in train_df['out']])))))
train_df.drop(drop_train)
val_df = val_dict[pos] #full_dict[pos].drop(train_df.index)

input_shape = np.shape(train_df.columns[:-1])

model = tf.keras.Sequential([

    tf.keras.layers.InputLayer(shape=input_shape, batch_size=256, dtype=None, 
                               sparse=None, batch_shape=None, input_tensor=None, 
                               name=None),
    tf.keras.layers.Conv1D(filters=40, kernel_size=4, strides=1),
    # tf.keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    # tf.keras.layers.Dense(units=128, activation='leaky_relu',
    #                       input_shape=input_shape),
    tf.keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    tf.keras.layers.Dense(units=32, activation='leaky_relu'),
    tf.keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    #tf.keras.layers.Dropout(0.1, noise_shape=None, seed=None),
    tf.keras.layers.Dense(units=1, activation='leaky_relu') #sigmoid')
])

model.compile(optimizer='adam', loss='mse') #'binary_crossentropy') #'mse')

model.summary()

# val_round = np.array([1 if val_out > 60 else 0 for val_out in list(val_df['out'])])
# train_round = np.array([1 if train_out > 60 else 0 for train_out in list(train_df['out'])])

val_round = np.array([val_out for val_out in list(val_df['out'])])
train_round = np.array([train_out for train_out in list(train_df['out'])])

losses = model.fit(np.array(train_df[train_df.columns[:-1]]).reshape(len(train_df), 4, 32), train_round, validation_data=(np.array(val_df[val_df.columns[:-1]]).reshape(len(val_df), 4, 32), val_round), batch_size=256, epochs=30)

loss_df = pd.DataFrame(losses.history)

loss_df.loc[:,['loss','val_loss']].plot()

pred = model.predict(val_df[train_df.columns[:-1]])

# plt.figure(figsize=(10,5))
# plt.scatter(val_df['out'], pred, alpha=0.1)
# plt.xlabel('True')
# plt.ylabel('Prediction')
# plt.show()

pred_round = [round(pred_[0]) for pred_ in pred]

plt.figure(figsize=(10,5))
plt.scatter(val_round, pred, alpha=0.1)
plt.xlabel('True')
plt.ylabel('Prediction')
plt.show()

print (sum([abs(pred_round[i] - val_round[i]) for i in range(len(pred))])/len(pred))

ValueError: Input 0 of layer "conv1d" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (256, 128)

In [ ]:
train_input = []
for i in tqdm(range(len(train_df))):
    train_input.append(np.array(train_df[train_df.columns[:-1]])[i].reshape(4, 32))

val_input = []
for i in tqdm(range(len(val_df))):
    val_input.append(np.array(val_df[val_df.columns[:-1]])[i].reshape(4, 32))

In [6]:
np.array(train_df[train_df.columns[:-1]])[1]

NameError: name 'train_df' is not defined

In [6]:
input_shape

(128,)

In [1]:
train_df.columns

NameError: name 'train_df' is not defined

In [13]:
np.array(val_df[val_df.columns[:-1]]).reshape(len(val_df), 4, 32)[0]

array([[ 0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., 67., 69.,
        68., 68., 84., 81., 84., 83.],
       [ 1.,  0.,  0.,  0.,  0., 14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  4., 67., 69.,
        68., 68., 87., 87., 84., 85.],
       [ 0.,  0.,  0.,  0.,  0., 24.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1., 67., 69.,
        68., 68., 77., 77., 76., 77.],
       [ 0.,  0.,  0.,  0.,  0., 32.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 67., 69.,
        68., 68., 76., 78., 78., 77.]])